In [1]:
import numpy as np
from tensorflow.keras.datasets import cifar10
from skimage.util import view_as_windows

from framework.pixelhop import *
from framework.utils import *

from skimage.measure import block_reduce
import xgboost as xgb
import warnings, gc
import time
import matplotlib.pyplot as plt

from tqdm import tqdm

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.__version__)

import os
print(os.getcwd())

from framework.dftloss import *

import lightgbm as lgb
import math
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix
import pandas as pd
import seaborn as sn


# ---------- Load MNIST data and split ----------
(x_train, y_train), (x_test,y_test) = cifar10.load_data()
# -----------Data Preprocessing-----------
x_train = np.asarray(x_train,dtype='float32')
x_test = np.asarray(x_test,dtype='float32')
y_train = np.asarray(y_train,dtype='int')
y_test = np.asarray(y_test,dtype='int')
print("training with input:", x_train.shape)


True
1
1.13.0
/media/hongyu/SSD/SSDUBUNTU/WUSL/Pixelhop-Pytorch
training with input: (50000, 32, 32, 3)


In [7]:
SaabArgs = [
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':False, 'cw': False}, # PQR transform in this step
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'cw': True},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'cw': True},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'cw': True},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'cw': True},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'cw': True},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'cw': True},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'cw': True},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'cw': True},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'cw': True},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'cw': True},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'cw': True},
            # {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'cw': True},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'cw': True}
            ]

shrinkArgs = [
            # {'func':Shrink, 'win':1, 'pad':true/false, 'stride': 1},
            {'func':NoShrink, 'win':1, 'pad':0, 'stride': 1, 'pooling':0},#PQR
            {'func':Shrink, 'win':3, 'pad':0, 'stride': 1, 'pooling':1,'poolingParms' :(3,2,1)},#conv1
            {'func':Shrink, 'win':3, 'pad':0, 'stride': 1, 'pooling':1,'poolingParms' :(3,2,1)},#conv2
            {'func':Shrink, 'win':3, 'pad':0, 'stride': 1, 'pooling':1,'poolingParms' :(3,2,1)},#conv3
            {'func':Shrink, 'win':3, 'pad':0, 'stride': 1, 'pooling':0,'poolingParms' :(3,2,1)},#conv4
            # {'func':Shrink, 'win':5, 'pad':0, 'stride': 1, 'pooling':0},#conv3
            ]

concatArg = {'func':Concat}



In [10]:
threshold1 = 0.0001 # for splitting, th2 <= kernels <= th1 will be treated as leaf nodes
threshold2 = 0.00005 # for feature selections, discard these kernels directly after each saab training
depth = 6

t1 = time.time()

# mymodel = Pixelhop(depth=depth, TH1=threshold1, TH2=threshold2, \
#     SaabArgs=SaabArgs, shrinkArgs=shrinkArgs, concatArg=concatArg, DCAC= True)

mymodel = Pixelhop(depth=depth, TH1=threshold1, TH2=threshold2, \
    SaabArgs=SaabArgs, shrinkArgs=shrinkArgs, concatArg=concatArg, DCAC= False)
mymodel.fit(x_train)
print(time.time() - t1)

       <WARNING> Too few 'SaabArgs/shrinkArgs' to get depth 6,             actual depth: 5
=====>c/w Saab Train Hop 0
PQR energy: [0.91086739 0.07377977 0.01535284]
check energy: [0.91086739 0.07377977 0.01535284]
FEATURE SHAPE in Hop0 [50000, 32, 32, 3]
=============================================>c/w Saab Train Hop 1
check input featmap: (50000, 32, 32, 3)
in cwsaab_n_layer, current #Saab: 1
check input X: (3, 50000, 32, 32)
in channel 0 Energy: 3
in cwsaab_layer_ 1
output: (50000, 30, 30, 20)
CHECK FEATURE SHAPE [50000, 15, 15, 20]
=============================================>c/w Saab Train Hop 2
check input featmap: (50000, 15, 15, 20)
in cwsaab_n_layer, current #Saab: 3
check input X: (20, 50000, 15, 15)
in channel 0 Energy: 9
in channel 1 Energy: 8
in channel 2 Energy: 6
in cwsaab_layer_ 2
output: (50000, 13, 13, 87)
CHECK FEATURE SHAPE [50000, 7, 7, 87]
=============================================>c/w Saab Train Hop 3
check input featmap: (50000, 7, 7, 87)
in cwsaab_n_layer, 

In [15]:
# save model
import os
os.makedirs('./model/',exist_ok = True)

trainname = 'max_abs_pooling_2ths'
module_filename = './model/model_' + trainname +'.pkl'
pickle.dump(mymodel,open(module_filename ,'wb'))
print("model saved at ", module_filename)

model saved at  ./model/model_max_abs_pooling_2ths.pkl


In [13]:
test = x_train[:1]
featmaps = mymodel.transform(test)
for featmap in featmaps:
       
    res = 1
    for s in featmap.shape:
        res *= s
    # print(featmap.shape, res)
    print(res)

3072
4500
4263
2196
414
